# Week Validation — Predictions vs Actuals

Evaluate **model probabilities** against **actual outcomes** for a week.

**It will:**
1) Load merged predictions (edges) and the weekly stats parquet
2) Normalize markets and players
3) Compute binary outcome per prop (exclude pushes)
4) Report **Brier**, **Log loss**, and **calibration** by market


## 0) Parameters

In [1]:
SEASON = 2025
WEEK   = 2
PATH_EDGES  = f'data/props/props_with_model_week{WEEK}.csv'
PATH_WEEKLY = f'data/weekly_player_stats_{SEASON}.parquet'
print('Using:', PATH_EDGES, PATH_WEEKLY)


Using: data/props/props_with_model_week2.csv data/weekly_player_stats_2025.parquet


## 1) Imports & helpers

In [2]:
import re, math, numpy as np, pandas as pd
try:
    from common_markets import standardize_input as _std_input
except Exception:
    def _norm_side(s: str) -> str:
        s = str(s).strip().lower()
        if 'over' in s: return 'over'
        if 'under' in s: return 'under'
        if s in ('yes','no'): return s
        return s
    ALIAS = {
        'player_anytime_td':'anytime_td',
        'player_reception_yds':'recv_yds', 'player_receptions':'receptions',
        'player_rush_yds':'rush_yds', 'player_rush_attempts':'rush_attempts',
        'player_pass_yds':'pass_yds', 'player_pass_attempts':'pass_attempts',
        'player_pass_completions':'pass_completions', 'player_pass_tds':'pass_tds',
        'player_pass_interceptions':'interceptions', 'pass_interceptions':'interceptions',
        'interceptions_thrown':'interceptions', 'ints':'interceptions', 'interception':'interceptions',
    }
    def _std_name(s):
        s = re.sub(r"[^a-z0-9\s]","",str(s).lower())
        s = re.sub(r"\s+"," ",s).strip()
        return s
    def _std_input(df: pd.DataFrame) -> pd.DataFrame:
        df = df.copy()
        src = next((c for c in ['market_std','market','market_name','market_key','key','market_slug'] if c in df.columns), None)
        if src is not None:
            ms = df[src].astype(str).str.lower().str.strip()
            df['market_std'] = ms.map(lambda x: ALIAS.get(x, x.replace('player_','')))
        else:
            df['market_std'] = ''
        # side
        s = None
        for c in ('name','side','selection','bet_name','over_under','bet_side'):
            if c in df.columns: s = df[c]; break
        if s is not None:
            df['name'] = s.astype(str).map(_norm_side)
        elif 'name' not in df.columns:
            df['name'] = pd.NA
        # point
        p = None
        for c in ('point','line','odds_point','bet_line','points'):
            if c in df.columns: p = df[c]; break
        if p is not None:
            df['point'] = pd.to_numeric(p, errors='coerce')
        elif 'point' not in df.columns:
            df['point'] = pd.NA
        # player
        pn = None
        for c in ('player','player_name','athlete','athlete_name','name_player'):
            if c in df.columns: pn = df[c]; break
        if pn is not None:
            s2 = pn.astype(str).map(_std_name)
            df['name_std'] = s2
            df['player_key'] = s2.str.replace(' ','-', regex=False)
        else:
            if 'name_std' not in df.columns: df['name_std'] = pd.NA
            if 'player_key' not in df.columns: df['player_key'] = pd.NA
        return df

def brier(y_true, p):
    y_true = np.asarray(y_true, float); p = np.asarray(p, float)
    return np.mean((p - y_true)**2)
def logloss(y_true, p, eps=1e-9):
    y_true = np.asarray(y_true, float); p = np.clip(np.asarray(p, float), eps, 1-eps)
    return -np.mean(y_true*np.log(p) + (1-y_true)*np.log(1-p))
def calib_table(y_true, p, bins=10):
    idx = np.clip((p * bins).astype(int), 0, bins-1)
    df = pd.DataFrame({'bin':idx, 'p':p, 'y':y_true})
    out = df.groupby('bin').agg(n=('y','size'), p_avg=('p','mean'), y_rate=('y','mean')).reset_index()
    out['abs_gap'] = (out['p_avg'] - out['y_rate']).abs()
    return out


## 2) Load predictions & weekly actuals

In [3]:
edges  = pd.read_csv(PATH_EDGES, low_memory=False)
weekly = pd.read_parquet(PATH_WEEKLY)
print('edges:', edges.shape, 'weekly:', weekly.shape)
edges = _std_input(edges)
weekly.columns = [str(c).strip().lower() for c in weekly.columns]
weekly_wk = weekly[(weekly.get('season',0)==SEASON) & (weekly.get('week',0)==WEEK)].copy() if 'season' in weekly.columns and 'week' in weekly.columns else weekly.copy()
print('weekly (filtered):', weekly_wk.shape)


edges: (14739, 24) weekly: (1142, 114)
weekly (filtered): (71, 114)


## 3) Map markets to stat columns

In [4]:
MARKET_COLS = {
  'recv_yds':        ['receiving_yards','rec_yds','rec_yards'],
  'receptions':      ['receptions','rec'],
  'rush_yds':        ['rushing_yards','rush_yds'],
  'rush_attempts':   ['rushing_attempts','rush_att'],
  'pass_yds':        ['passing_yards','pass_yds'],
  'pass_attempts':   ['passing_attempts','attempts'],
  'pass_completions':['completions','pass_completions','passing_completions'],
  'pass_tds':        ['passing_tds','pass_tds'],
  'interceptions':   ['interceptions','ints'],
  'anytime_td':      ['_any_td_dummy'],
}
td_rush = weekly_wk['rushing_tds'] if 'rushing_tds' in weekly_wk.columns else 0
td_rec  = weekly_wk['receiving_tds'] if 'receiving_tds' in weekly_wk.columns else 0
weekly_wk['_any_td_dummy'] = ((pd.Series(td_rush).fillna(0) + pd.Series(td_rec).fillna(0)) >= 1).astype(int)
actual_col = {m: next((c for c in cands if c in weekly_wk.columns), None) for m, cands in MARKET_COLS.items()}
actual_col


{'recv_yds': 'receiving_yards',
 'receptions': 'receptions',
 'rush_yds': 'rushing_yards',
 'rush_attempts': None,
 'pass_yds': 'passing_yards',
 'pass_attempts': 'attempts',
 'pass_completions': 'completions',
 'pass_tds': 'passing_tds',
 'interceptions': None,
 'anytime_td': '_any_td_dummy'}

## 4) Build per-player actuals for this week

In [5]:
def _std_name(s):
    s = re.sub(r'[^a-z0-9\s]','',str(s).lower())
    s = re.sub(r'\s+',' ',' '.join(s.split())).strip()
    return s
pname_col = next((c for c in ['player','player_name','name'] if c in weekly_wk.columns), None)
weekly_wk['name_std'] = weekly_wk[pname_col].map(_std_name) if pname_col else ''
keep_cols = ['name_std']
for c in actual_col.values():
    if c:
        keep_cols.append(c)
keep_cols = [c for c in keep_cols if c in weekly_wk.columns]
wk = weekly_wk[keep_cols].drop_duplicates('name_std').copy()
wk.shape


(71, 9)

## 5) Join predictions to actuals and compute outcomes

In [6]:
df = edges.copy()
df['market_std'] = df['market_std'].astype(str).str.lower()
df['name'] = df['name'].astype(str).str.lower().str.strip()
df['point'] = pd.to_numeric(df.get('point', np.nan), errors='coerce')
id_col = 'name_std' if 'name_std' in df.columns and 'name_std' in wk.columns else None
if id_col is None:
    raise RuntimeError('Could not find common identity column (name_std).')
df = df.merge(wk, on=id_col, how='left', suffixes=('','_actual'))
import numpy as np, pandas as pd
def row_actual(r):
    m = r['market_std']; a_col = actual_col.get(m)
    if not a_col: return np.nan, True
    val = r.get(a_col)
    if pd.isna(val): return np.nan, True
    side = r['name']
    if m == 'anytime_td':
        y = int(val >= 1)
        return (1 if side == 'yes' else 0) if side in ('yes','no') else np.nan, False if side in ('yes','no') else True
    if pd.isna(r['point']): return np.nan, True
    if side == 'over':
        if val == r['point']: return np.nan, True
        return int(val > r['point']), False
    if side == 'under':
        if val == r['point']: return np.nan, True
        return int(val < r['point']), False
    return np.nan, True
out = df.apply(lambda r: row_actual(r), axis=1, result_type='expand')
df['actual'] = out[0]; df['drop_row'] = out[1].fillna(True)
scored = df[(~df['drop_row']) & df['model_prob'].notna() & df['actual'].notna()].copy()
print('Scored rows:', len(scored), 'of', len(df))
scored.head(3)


Scored rows: 0 of 14739


,game_id,commence_time,home_team,away_team,game,bookmaker,bookmaker_title,market,market_std,player,...,receiving_yards,receptions,rushing_yards,passing_yards,attempts,completions,passing_tds,_any_td_dummy,actual,drop_row


## 6) Metrics by market

In [7]:
def brier(y_true, p):
    y_true = np.asarray(y_true, float); p = np.asarray(p, float)
    return np.mean((p - y_true)**2)
def logloss(y_true, p, eps=1e-9):
    y_true = np.asarray(y_true, float); p = np.clip(np.asarray(p, float), eps, 1-eps)
    return -np.mean(y_true*np.log(p) + (1-y_true)*np.log(1-p))
def agg_market(g):
    p = g['model_prob'].astype(float).values
    y = g['actual'].astype(float).values
    import pandas as pd, numpy as np
    return pd.Series({'n':len(g), 'avg_pred':np.mean(p), 'base_rate':np.mean(y), 'brier':brier(y,p), 'logloss':logloss(y,p), 'abs_calib_gap':abs(np.mean(p)-np.mean(y))})
report = (scored.groupby('market_std', as_index=False).apply(agg_market)
          .sort_values(['brier','logloss','n'], ascending=[True,True,False]))
report


/var/folders/zm/h0vxhn1x3vz5nph_c0qc60t00000gn/T/ipykernel_2707/3924332218.py:12: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  report = (scored.groupby('market_std', as_index=False).apply(agg_market)


KeyError: 'brier'

## 7) Calibration tables (deciles)

In [8]:
def calib_table(y_true, p, bins=10):
    import pandas as pd, numpy as np
    idx = np.clip((p * bins).astype(int), 0, bins-1)
    df = pd.DataFrame({'bin':idx, 'p':p, 'y':y_true})
    out = df.groupby('bin').agg(n=('y','size'), p_avg=('p','mean'), y_rate=('y','mean')).reset_index()
    out['abs_gap'] = (out['p_avg'] - out['y_rate']).abs()
    return out
examples = (report.sort_values('n', ascending=False)['market_std'].head(3).tolist() if len(report) else [])
tables = {}
for m in examples:
    sub = scored[scored['market_std']==m]
    tables[m] = calib_table(sub['actual'].values.astype(float), sub['model_prob'].values.astype(float), bins=10)
tables


NameError: name 'report' is not defined